# further pretraining LUKE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForMaskedLM, Trainer, TrainingArguments
import torch
from transformers import AdamW
import datasets


In [ ]:
'''
# Load your custom dataset
dataset_name = "eurlex"
dataset = load_dataset(dataset_name)
train_dataset = dataset["train"]
test_dataset = dataset["test"]

# Check the type of the dataset
print(type(train_dataset))
print(type(test_dataset))
,,,

In [ ]:
#development_data = "/content/drive/MyDrive/eurlex_dataset/eurlex_train"
dataset = datasets.load_from_disk("/content/drive/MyDrive/eurlex_dataset/eurlex_train")

In [ ]:
print(dataset)

Dataset({
    features: ['celex_id', 'title', 'text', 'eurovoc_concepts'],
    num_rows: 45000
})


In [ ]:
# List to store dictionaries for each row
dataset_list = []

# Iterate over the dataset
for example in dataset:
    # Create a dictionary for the current row
    row_dict = {
        'celex_id': example['celex_id'],
        'title': example['title'],
        'text': example['text'],
        'eurovoc_concepts': example['eurovoc_concepts']
    }

    # Append the dictionary to the list
    dataset_list.append(row_dict)


In [ ]:
print(len(dataset_list))

45000


In [ ]:
# Calculate the split indices
split_index1 = len(dataset_list) // 4
split_index2 = split_index1 * 2
split_index3 = split_index1 * 3

# Divide the dataset list into four parts
dataset_part1 = dataset_list[:split_index1]
dataset_part2 = dataset_list[split_index1:split_index2]
dataset_part3 = dataset_list[split_index2:split_index3]
dataset_part4 = dataset_list[split_index3:]

# Print the number of rows in each part
print("Number of rows in part 1:", len(dataset_part1))
print("Number of rows in part 2:", len(dataset_part2))
print("Number of rows in part 3:", len(dataset_part3))
print("Number of rows in part 4:", len(dataset_part4))



Number of rows in part 1: 11250
Number of rows in part 2: 11250
Number of rows in part 3: 11250
Number of rows in part 4: 11250


In [ ]:
print(split_index1)

11250


In [ ]:
from transformers import LukeTokenizer, LukeForMaskedLM, LukeConfig
from torch.utils.data import DataLoader

In [ ]:
# Create a LukeTokenizer
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-large")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

entity_vocab.json:   0%|          | 0.00/15.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/33.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/839 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import LukeTokenizer

# Load LukeTokenizer
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-large")

# Check if the tokenizer has a "mask_token_id"
if hasattr(tokenizer, "mask_token_id"):
    print("The tokenizer has a 'mask_token_id':", tokenizer.mask_token_id)
else:
    print("The tokenizer does not have a 'mask_token_id'")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


The tokenizer has a 'mask_token_id': 50264


In [ ]:
# Print the first few samples in the train_dataset
#dataset_part1
#dataset_part2

for i in range(5):
    print(dataset_part1[i])

{'celex_id': '32014R0727', 'title': "Commission Implementing Regulation (EU) No 727/2014 of 30 June 2014 initiating a ‘new exporter’ review of Council Implementing Regulation (EU) No 1389/2011 imposing a definitive anti-dumping duty on imports of trichloroisocyanuric acid originating in the People's Republic of China, repealing the duty with regard to imports from one exporter in this country and making these imports subject to registration\n", 'text': "1.7.2014 EN Official Journal of the European Union L 192/42\nCOMMISSION IMPLEMENTING REGULATION (EU) No 727/2014\nof 30 June 2014\ninitiating a ‘new exporter’ review of Council Implementing Regulation (EU) No 1389/2011 imposing a definitive anti-dumping duty on imports of trichloroisocyanuric acid originating in the People's Republic of China, repealing the duty with regard to imports from one exporter in this country and making these imports subject to registration\nTHE EUROPEAN COMMISSION\n,\nHaving regard to the Treaty on the Functio

In [ ]:
# Access the text data from the dataset
texts_train = [sample["text"] for sample in dataset_part1]


# Tokenize your text data
tokenized_data = tokenizer(texts_train, padding=True, truncation=True, return_tensors="pt")


In [ ]:
# Assuming tokenized_data contains your tokenized data
for i in range(5):  # Print the first 5 tokenized samples
    print(f"Sample {i + 1} - Input IDs: {tokenized_data['input_ids'][i]}")
    print(f"Sample {i + 1} - Attention Mask: {tokenized_data['attention_mask'][i]}")
    print()

Sample 1 - Input IDs: tensor([    0,   134,     4,   406,     4, 16310, 13245, 13485,  3642,     9,
            5,   796,  1332,   226, 27886,    73,  3714, 50118, 10370,   448,
        41336, 28922,  3850, 12613,  1862, 26747,  6597,  6034,    36, 14707,
           43,   440,   262,  2518,    73, 16310, 50118,  1116,   389,   502,
          777, 50118,   179,  4933,  1295,    10,    44,   711,  4651,  4553,
        10679,    17,    27,  1551,     9,  1080, 42400,   154, 18912,    36,
        14707,    43,   440,   508,  5046,    73, 22748, 13223,    10, 17756,
         1475,    12,   417, 14672,  4053,    15,  5058,     9,  2664,  1725,
        10159, 12992,  1975,  7010,   710,   636, 10395, 34864,    11,     5,
         1806,    18,  3497,     9,   436,     6, 27792,     5,  4053,    19,
         6203,     7,  5058,    31,    65,  4553, 10679,    11,    42,   247,
            8,   442,   209,  5058,  2087,     7,  5770, 50118, 13354, 10353,
        23075,  1889, 31299, 41336, 50118,

In [ ]:
# Create PyTorch Dataset using CustomDataset class
class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask):
        self.input_ids = input_ids
        self.attention_mask = attention_mask

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
        }


In [ ]:
# Create PyTorch Dataset using CustomDataset class
custom_dataset = CustomDataset(tokenized_data["input_ids"], tokenized_data["attention_mask"])

In [ ]:
# Define your batch size
your_batch_size = 16
learning_rate = 1e-5

In [ ]:
# Create a DataLoader
data_loader = DataLoader(custom_dataset, batch_size=your_batch_size, shuffle=True)

In [ ]:
# Define Luke model configuration and load the pre-trained LukeForMaskedLM model
config = LukeConfig.from_pretrained("studio-ousia/luke-large")
model = LukeForMaskedLM.from_pretrained("studio-ousia/luke-large", config=config)

pytorch_model.bin:   0%|          | 0.00/2.25G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
# Load the previously saved checkpoint
checkpoint_path = "/content/drive/MyDrive/NLP/luke_large_checkpoints/luke_large_model_checkpoint_epoch_4.pth"
checkpoint = torch.load(checkpoint_path)

# Load the model state_dict
model.load_state_dict(checkpoint)

# Set the model to training mode
model.train()

LukeForMaskedLM(
  (luke): LukeModel(
    (embeddings): LukeEmbeddings(
      (word_embeddings): Embedding(50267, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (entity_embeddings): LukeEntityEmbeddings(
      (entity_embeddings): Embedding(500000, 256, padding_idx=0)
      (entity_embedding_dense): Linear(in_features=256, out_features=1024, bias=False)
      (position_embeddings): Embedding(514, 1024)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LukeEncoder(
      (layer): ModuleList(
        (0-23): 24 x LukeLayer(
          (attention): LukeAttention(
            (self): LukeSelfAttention(
              (query): Linear(in_

In [ ]:
# Train the model
optimizer = AdamW(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from tqdm import tqdm
import os

# Train the model
num_epochs = 20 # Set the number of epochs 40
mask_probability = 0.15
accumulation_steps = 4  # Define the accumulation steps
checkpoint_dir = "/content/drive/MyDrive/NLP/luke_large_checkpoints"  # Specify the directory to save the checkpoints

# Create the directory if it doesn't exist
os.makedirs(checkpoint_dir, exist_ok=True)

for epoch in range(4, num_epochs + 1):
    for batch_idx, batch in enumerate(tqdm(data_loader, desc=f"Epoch {epoch + 1}/{num_epochs}")):
        inputs = batch["input_ids"]
        attention_mask = batch["attention_mask"]

        mask_indices = torch.rand(inputs.shape) < mask_probability
        labels = inputs.clone()

        inputs[mask_indices] = tokenizer.mask_token_id

        outputs = model(inputs, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss = loss / accumulation_steps
        loss.backward()

        if (batch_idx + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

    # Save model checkpoint after each epoch with epoch number in the file name
    checkpoint_path = os.path.join(checkpoint_dir, f"luke_large_model_checkpoint_epoch_{epoch + 1}.pth")
    torch.save(model.state_dict(), checkpoint_path)

    # Save the further pretrained model
    model.save_pretrained("/content/drive/MyDrive/NLP/luke_large_eurlex_model")

Epoch 5/20:  99%|█████████▉| 699/704 [23:55:01<10:14, 122.83s/it]

In [ ]:
#To later load the pretrained Luke model for downstream tasks

from transformers import LukeForMaskedLM, LukeTokenizer

# Load LukeTokenizer
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-base")

# Load the saved model
model = LukeForMaskedLM.from_pretrained("/content/drive/MyDrive/NLP/luke_eurlex_model")

# Now you can use the loaded model for downstream tasks
# For example, you can use it for text generation, classification, etc.
